In [63]:
import pandas as pd
import numpy as np

data = {
    'users': [1, 2, 3, 4, 5, 6],
    '1': [1, None, 2, None, None, 1],
    '2': [None, None, 4, 2, None, None],
    '3': [3, 5, None, 4, 4, 3],
    '4': [None, 4, 1, None, 3, None],
    '5': [None, None, 2, 5, 4, 3],
    '6': [5, None, None, None, 2, None],
    '7': [None, 4, 3, None, None, None],
    '8': [None, None, None, 4, None, 2],
    '9': [5, None, 4, None, None, None],
    '10': [None, 2, 3, None, None, None],
    '11': [4, 1, 5, 2, 2, 4],
    '12': [None, 3, None, None, 5, None]
}

df = pd.DataFrame(data)
print("Original Matrix:\n", df.to_string())

df_centered = df.copy()

cols_to_center = df_centered.columns[1:]

row_avg = df_centered[cols_to_center].mean(axis=1)

for index, row in df_centered.iterrows():
    non_empty_values = row.dropna()  # Exclude NaN values
    avg = row_avg[index]
    df_centered.loc[index, non_empty_values.index] -= avg

df_centered.fillna(0, inplace=True)

df_centered['row_avg'] = row_avg

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0
    return dot_product / (norm_a * norm_b)
similarities = []
first_row = df_centered.iloc[0, 1:-1]
for i in range(1, len(df_centered)):  # Start from index 1 to exclude the first row
    a = first_row
    b = df_centered.iloc[i, 1:-1]
    similarities.append(cosine_similarity(a, b))
df_centered['Similarities'] = [np.nan] + similarities

print("\nUpdated Matrix with Row Similarities with First Row:")
print(df_centered.to_string())
positive_similarities_indices = [i for i, sim in enumerate(similarities, start=1) if sim > 0]
s_1j_positive = [similarities[i-1] for i in positive_similarities_indices]
r_j5_values = df.iloc[positive_similarities_indices, 5].dropna()
numerator = sum(s * r for s, r in zip(s_1j_positive, r_j5_values))
r_15 = numerator / sum(s_1j_positive)
print("\nValue of the element at (row 1, column 5):", r_15)

Original Matrix:
    users    1    2    3    4    5    6    7    8    9   10  11   12
0      1  1.0  NaN  3.0  NaN  NaN  5.0  NaN  NaN  5.0  NaN   4  NaN
1      2  NaN  NaN  5.0  4.0  NaN  NaN  4.0  NaN  NaN  2.0   1  3.0
2      3  2.0  4.0  NaN  1.0  2.0  NaN  3.0  NaN  4.0  3.0   5  NaN
3      4  NaN  2.0  4.0  NaN  5.0  NaN  NaN  4.0  NaN  NaN   2  NaN
4      5  NaN  NaN  4.0  3.0  4.0  2.0  NaN  NaN  NaN  NaN   2  5.0
5      6  1.0  NaN  3.0  NaN  3.0  NaN  NaN  2.0  NaN  NaN   4  NaN

Updated Matrix with Row Similarities with First Row:
      users    1    2         3         4         5         6         7    8    9        10        11        12   row_avg  Similarities
0 -2.600000 -2.6  0.0 -0.600000  0.000000  0.000000  1.400000  0.000000  0.0  1.4  0.000000  0.400000  0.000000  3.600000           NaN
1 -1.166667  0.0  0.0  1.833333  0.833333  0.000000  0.000000  0.833333  0.0  0.0 -1.166667 -2.166667 -0.166667  3.166667     -0.178542
2  0.000000 -1.0  1.0  0.000000 -2.000000 -1